<h1><font color = #fc7cc9> Ch. 10 Data Aggregation and Group Operations
    <br>pg. 287 - 316</h1>
<p> 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns



<h2> <font color = #39abed> 10.1 GroupBy Mechanics
    </h2>
    <p> 

<h3> <font color = #39abed> Iterating Over Groups
    </h3>
 <p>

<h3> <font color = #39abed> Selecting a Column or Subset of Columns
    </h3>
 <p>

<h3> <font color = #39abed> Grouping with Dicts and Series
    </h3>
 <p>

<h3> <font color = #39abed> Grouping with Functions
    </h3>
 <p>

<h3> <font color = #39abed> Grouping by Index Levels
    </h3>
 <p>

<h2> <font color = #39abed> 10.2 Data Aggregation
    </h2>
    <p> 

<h3> <font color = #39abed> Column-Wise Multiple Function Application
    </h3>
 <p>

<h3> <font color = #39abed>Returning Aggregated Data Without Row Indexes
    </h3>
 <p>

<h2> <font color = #39abed> 10.3 Apply: General split-apply-combine
    </h2>
    <p> 

<h3> <font color = #39abed> Suppressing Group Keys
    </h3>
 <p>

<h3> <font color = #39abed> Quantile and Bucket Analysis
    </h3>
 <p>

<h3> <font color = #39abed>Example: Filling Missing Values with Group-Specific Values
    </h3>
 <p>

<h3> <font color = #39abed> Example: Random Sampling and Permutation
    </h3>
 <p>

<h3> <font color = #39abed> Example: Group Weighted Average and Correlation
    </h3>
 <p>

<h3> <font color = #39abed> Example: Group-Wise Linear Regression
    </h3>
 <p>

<h2> <font color = #39abed> 10.4 Pivot Tables and Cross-Tabulations
    </h2>
    <p> 

<h3> <font color = #39abed> Cross-Tabulations: Crosstab
    </h3>
 <p>

<h2> <font color = #39abed> 10.5 Conclusion
    </h2>
    <p> 